# Graph Analytics

This notebook is dedicated to provide some insights on the graph, answering the following questions:
- What is the percentage of schools with rating of 2 or higher?
- Which schools have the highest number of nearby Lines nearby / in the area?
- Are there any schools which do not have any Line nearby 600m?
- What's the average rating inside and outside of the first 9 districts?

## Imports

In [33]:
from neomodel import db, config
from credentials import Credentials
from models import *
import json
from shapely.geometry import Point, Polygon
from pyproj import Transformer

config.DATABASE_URL = Credentials.getNeo4JDatabaseURI()

file_path = '../data/polygon-wien-bezirksgrenzen.json'
with open(file_path, 'r') as f:
    polygon_data = json.load(f)

## Percentage of schools with rating 2 or higher

In [34]:
query = """
        MATCH (s:School)
        WITH count(s) AS total_schools
        MATCH (s:School)
        WHERE s.rating >= 2
        WITH total_schools, count(s) AS schools_with_rating_2_or_higher
        RETURN (toFloat(schools_with_rating_2_or_higher) / total_schools) * 100 AS percentage_of_schools_with_rating_2_or_higher
        """
results, _ = db.cypher_query(query)

results

[[51.61290322580645]]

## Schools with highest number of lines nearby / in the area

In [40]:
query = """
        MATCH (s:School)-[r:IS_IN_AREA]->() 
        WITH s, count(r) AS in_area_connections
        MATCH (s)-[r:IS_NEARBY]->()
        WITH s, in_area_connections, count(r) AS nearby_connections
        RETURN s.name, (in_area_connections + nearby_connections) AS total_connections
        ORDER BY total_connections DESC
        LIMIT 5
        """
results, _ = db.cypher_query(query)

results

[['Gastgewerbefachschule des Schulvereins der Wiener Gastwirte', 22],
 ['PVS Judenplatz 6', 22],
 ['VS Stolberggasse 53', 21],
 ['VS Börsegasse 5', 21],
 ['Höhere Bundeslehr- und Versuchsanstalt für Textilindustrie und Informatik',
  21]]

## Non-connected schools

In [36]:
query = """
        MATCH (s:School)
        WHERE NOT (s)-[:IS_IN_AREA]->() AND NOT (s)-[:IS_NEARBY]->()
        RETURN s.name
        """
results, _ = db.cypher_query(query)

results

[['MS 22, Sonnenallee 116'],
 ['VS Hannah-Arendt-Platz 8'],
 ['VS 22, Sonnenallee 116'],
 ['BgBRg Maria-Trapp-Platz 5']]

## Average ratings inside and outside of the first 9 districts

In [37]:
# Transformer because the coordinates of polygon-wien-bezirksgrenzen.json are in a different format
transformer = Transformer.from_crs("EPSG:4326", "EPSG:31256", always_xy=True)

# Extract coordinates from first 9 districts
district_polygons = []
for feature in polygon_data['features']:
    district_number = feature['properties']['BEZNR']
    if district_number < 10:
        coordinates = feature['geometry']['coordinates'][0]
        polygon = Polygon(coordinates)
        district_polygons.append((district_number, polygon))

def is_school_in_first_9_district(school_latitude, school_longitude):
    transformed_longitude, transformed_latitude = transformer.transform(school_longitude, school_latitude)
    point = Point(transformed_longitude, transformed_latitude) 
    for district_number, polygon in district_polygons:
        if polygon.contains(point):
            return district_number
    return None


In [38]:
def calculate_average_rating_inside_outside(school_data):
    inside_ratings = []
    outside_ratings = []

    for school in school_data:
        school_latitude = school['latitude']
        school_longitude = school['longitude']
        school_rating = school['rating']

        inner_area = is_school_in_first_9_district(school_latitude, school_longitude)

        if inner_area:
            inside_ratings.append(school_rating)
        else:
            outside_ratings.append(school_rating)

    inside_average = sum(inside_ratings) / len(inside_ratings) if inside_ratings else None
    outside_average = sum(outside_ratings) / len(outside_ratings) if outside_ratings else None

    return inside_average, outside_average


In [39]:
query = """
        MATCH (s:School)
        RETURN s.location_latitude AS latitude, s.location_longitude AS longitude, s.rating AS rating
        """
results, _ = db.cypher_query(query)

school_data = [
    {"latitude": record[0], "longitude": record[1], "rating": record[2]}
    for record in results
]

inside_avg, outside_avg = calculate_average_rating_inside_outside(school_data)

inside_avg, outside_avg

(1.5566037735849056, 2.1818181818181817)